<a href="https://colab.research.google.com/github/hagusta/colab/blob/master/casava_interp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import os
drive.mount('/content/gdrive', force_remount=True)
kaggle_json='gdrive/MyDrive/kaggle.json'
model_dir='gdrive/MyDrive/casava'
working_dir='/dev/shm'

Mounted at /content/gdrive


In [2]:
!pip uninstall -y kaggle
!pip install -Uqq kaggle

#import fastai
#import torch
#import fastbook

Uninstalling kaggle-1.5.10:
  Successfully uninstalled kaggle-1.5.10
     |████████████████████████████████| 61kB 4.3MB/s 


In [3]:
!pip uninstall -y fastai
!pip install -Uqq fastai
!pip install -Uqq fastbook


Uninstalling fastai-1.0.61:
  Successfully uninstalled fastai-1.0.61
     |████████████████████████████████| 194kB 8.5MB/s 
     |████████████████████████████████| 61kB 7.1MB/s 
     |████████████████████████████████| 727kB 7.7MB/s 
     |████████████████████████████████| 51kB 6.0MB/s 
     |████████████████████████████████| 1.2MB 13.2MB/s 
     |████████████████████████████████| 51kB 6.7MB/s 


In [4]:
import os  
from pathlib import Path 
import pandas as pd
import json
import shutil
import fastai
import torch
import fastbook

In [5]:
!tar xzf gdrive/MyDrive/casava_data/casava.tar.gz --directory /dev/shm
df=pd.read_csv('/dev/shm/casava/train.csv')
#f=open('/dev/shm/casava/label_num_to_disease_map.json')
#label_name=dict(json.load(f))
label_name_dict=dict({'0':'CBB',
                 '1':'CBSD',
                 '2':'CGM',
                 '3':'CMD',
                 '4':'Healty'})
imgs=df.image_id.to_list()
lbl=df.label.to_list()
labels=[ label_name_dict[str(i)] for i in lbl]
label_dict=dict(zip(imgs,labels))

In [6]:
def get_label(o): return label_dict[o.name]

In [7]:
from fastai.vision.all import *

In [8]:
casava=DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    #get_y=lambda p: label_dict[p.name],
    #get_y=parent_label,
    get_y=get_label,
    #item_tfms=Resize(512,method=ResizeMethod.Squish),
    item_tfms=Resize(512),
    #batch_tfms=[*aug_transforms(size=224,mult=2),Normalize.from_stats(*imagenet_stats)]
    #batch_tfms=[Normalize.from_stats(*imagenet_stats)]
    batch_tfms=aug_transforms(size=224,mult=2)
    )

In [9]:
path=Path('/dev/shm/casava/train_images')
dls = casava.dataloaders(path,bs=16)

In [10]:
mdls=["casava_resnet18","casava_resnet18.1"]

In [11]:
learn = cnn_learner(dls,
                    models.resnet18,
                    metrics=error_rate)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


In [12]:
import pickle

In [13]:
interp=None
mdl_path=Path('/content/gdrive/MyDrive/casava/').absolute()
for m in mdls:
  learn=learn.load(mdl_path/m)
  if isinstance(interp, ClassificationInterpretation):
    del interp
  interp = ClassificationInterpretation.from_learner(learn)
  print(m,interp.print_classification_report())
  top50 = top=interp.top_losses(100)
  #top50=[]
  fname=str(m+'.top40.pkl')
  f=open(mdl_path/fname,mode='bw')
  pickle.dump(top50,f)

              precision    recall  f1-score   support

         CBB       0.61      0.28      0.38       218
        CBSD       0.49      0.46      0.47       454
         CGM       0.39      0.43      0.41       497
         CMD       0.93      0.75      0.83      2586
      Healty       0.39      0.83      0.53       524

    accuracy                           0.67      4279
   macro avg       0.56      0.55      0.52      4279
weighted avg       0.74      0.67      0.68      4279

casava_resnet18 None


              precision    recall  f1-score   support

         CBB       0.65      0.21      0.31       218
        CBSD       0.51      0.33      0.40       454
         CGM       1.00      0.01      0.02       497
         CMD       0.87      0.87      0.87      2586
      Healty       0.34      0.86      0.49       524

    accuracy                           0.68      4279
   macro avg       0.67      0.45      0.42      4279
weighted avg       0.77      0.68      0.65      4279

casava_resnet18.1 None


In [ ]:
interp=None
mdls=["casava_resnet18.1"]
mdl_path=Path('/content/gdrive/MyDrive/casava/').absolute()
for m in mdls:
  learn=learn.load(mdl_path/m)
  if isinstance(interp, ClassificationInterpretation):
    del interp
  interp = ClassificationInterpretation.from_learner(learn)
  print(m,interp.print_classification_report())
  top50 = top=interp.top_losses(100)
  #top50=[]
  fname=str(m+'.top40.pkl')
  f=open(mdl_path/fname,mode='bw')
  pickle.dump(top50,f)

              precision    recall  f1-score   support

         CBB       0.64      0.68      0.66       218
        CBSD       0.84      0.76      0.80       454
         CGM       0.94      0.36      0.52       497
         CMD       0.89      0.99      0.94      2586
      Healty       0.66      0.73      0.69       524

    accuracy                           0.84      4279
   macro avg       0.79      0.70      0.72      4279
weighted avg       0.85      0.84      0.83      4279

casava_resnet18.1 None


In [ ]:
top50s=[]
for m in mdls:
  fname=str(m+'.top40.pkl')
  f=open(mdl_path/fname,'br')
  top50=pickle.load(f)
  top50s=top50s+[dls.valid_ds.items[f] for f in sorted(top50[1].tolist())]
  #top50s.append(_top50)

In [ ]:
toplossimg=[]
_=[toplossimg.append(f) for f in sorted(top50s) if f not in toplossimg]

In [ ]:
len(toplossimg)


109

In [ ]:
learn.pred??

In [ ]:
toplossimg[:5]

[Path('/dev/shm/casava/train_images/CBB/1119847734.jpg'),
 Path('/dev/shm/casava/train_images/CBB/2044199243.jpg'),
 Path('/dev/shm/casava/train_images/CBB/2243019094.jpg'),
 Path('/dev/shm/casava/train_images/CBB/2519536403.jpg'),
 Path('/dev/shm/casava/train_images/CBB/2641912037.jpg')]

In [ ]:
learn=learn.load(mdl_path/"casava_resnet18")
if isinstance(interp, ClassificationInterpretation):
  del interp
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
print("casava_resnet18",interp.print_classification_report())

              precision    recall  f1-score   support

         CBB       0.53      0.73      0.61       218
        CBSD       0.85      0.72      0.78       454
         CGM       0.91      0.55      0.68       497
         CMD       0.92      0.97      0.94      2586
      Healty       0.67      0.72      0.69       524

    accuracy                           0.85      4279
   macro avg       0.78      0.74      0.74      4279
weighted avg       0.86      0.85      0.85      4279

casava_resnet18 None


In [ ]:
learn=learn.load(mdl_path/"casava_resnet18")
if isinstance(interp, ClassificationInterpretation):
  del interp
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
print("casava_resnet18",interp.print_classification_report())

              precision    recall  f1-score   support

         CBB       0.62      0.71      0.66       218
        CBSD       0.87      0.71      0.78       454
         CGM       0.92      0.51      0.65       497
         CMD       0.91      0.99      0.94      2586
      Healty       0.68      0.74      0.71       524

    accuracy                           0.86      4279
   macro avg       0.80      0.73      0.75      4279
weighted avg       0.86      0.86      0.85      4279

casava_resnet18 None


In [ ]:
learn=learn.load(mdl_path/"casava_resnet18.1")
if isinstance(interp, ClassificationInterpretation):
  del interp
interp = ClassificationInterpretation.from_learner(learn)
print("casava_resnet18",interp.print_classification_report())

              precision    recall  f1-score   support

         CBB       0.66      0.67      0.67       218
        CBSD       0.87      0.74      0.80       454
         CGM       0.91      0.54      0.68       497
         CMD       0.92      0.98      0.95      2586
      Healty       0.66      0.76      0.71       524

    accuracy                           0.86      4279
   macro avg       0.80      0.74      0.76      4279
weighted avg       0.87      0.86      0.86      4279

casava_resnet18 None


In [ ]:
learn=learn.load(mdl_path/"casava_resnet18.1")
if isinstance(interp, ClassificationInterpretation):
  del interp
interp = ClassificationInterpretation.from_learner(learn)
print("casava_resnet18",interp.print_classification_report())

              precision    recall  f1-score   support

         CBB       0.66      0.67      0.67       218
        CBSD       0.87      0.74      0.80       454
         CGM       0.91      0.54      0.68       497
         CMD       0.92      0.98      0.95      2586
      Healty       0.66      0.76      0.71       524

    accuracy                           0.86      4279
   macro avg       0.80      0.74      0.76      4279
weighted avg       0.87      0.86      0.86      4279

casava_resnet18 None


In [ ]:
!pip install timm

     |████████████████████████████████| 245kB 7.8MB/s 


In [ ]:
import timm

In [ ]:
model = timm.create_model('resnext50_32x4d', pretrained=true)

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/resnext50_32x4d_ra-d733960d.pth" to /root/.cache/torch/hub/checkpoints/resnext50_32x4d_ra-d733960d.pth


In [ ]:
model.fc=nn.Linear(2048,5)

In [ ]:
learn = Learner(dls,
                    model,
                    metrics=error_rate)

In [ ]:
learn=learn.load(mdl_path/"casava_resnext50_32x4d")
if isinstance(interp, ClassificationInterpretation):
  del interp
interp = ClassificationInterpretation.from_learner(learn)
print("casava_resnet18",interp.print_classification_report())

              precision    recall  f1-score   support

         CBB       0.00      0.00      0.00       218
        CBSD       0.52      0.03      0.06       454
         CGM       0.83      0.01      0.02       497
         CMD       0.61      1.00      0.76      2586
      Healty       0.70      0.04      0.08       524

    accuracy                           0.61      4279
   macro avg       0.53      0.22      0.18      4279
weighted avg       0.61      0.61      0.48      4279

casava_resnet18 None


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
learn=learn.load(mdl_path/"casava_resnext50_32x4d")
if isinstance(interp, ClassificationInterpretation):
  del interp
interp = ClassificationInterpretation.from_learner(learn)
print("casava_resnet18",interp.print_classification_report())

              precision    recall  f1-score   support

         CBB       0.43      0.01      0.03       218
        CBSD       0.27      0.26      0.27       454
         CGM       0.88      0.01      0.03       497
         CMD       0.73      0.47      0.57      2586
      Healty       0.19      0.78      0.30       524

    accuracy                           0.41      4279
   macro avg       0.50      0.31      0.24      4279
weighted avg       0.62      0.41      0.42      4279

casava_resnet18 None


In [ ]:
nn.CrossEntropyLoss??

In [ ]:
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=False)
target = torch.empty(3, dtype=torch.long).random_(5)
output = loss(input, target)

In [ ]:
output

tensor(1.4627)

In [ ]:
input

tensor([[-2.8521, -1.2051,  1.6213,  1.5941, -0.4810],
        [-0.6521,  0.3849, -0.5365,  0.3320, -0.3207],
        [ 0.5116,  0.7983, -0.1532, -0.1646,  0.2479]])

In [ ]:
target

tensor([2, 0, 0])

In [ ]:
learn.freeze?